# Tile mosaic image acquisitions from Opera Phenix 

This notebook exhibits the original tiling code used to compile Opera Phenix acquisition into contiguous mosaic images for tracking cells across tiles. 

### Original approach

In [1]:
from macrohet import dataio, tile

/home/dayn/analysis/macrohet/macrohet/tile.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [45]:
metadata = dataio.read_harmony_metadata('../data/untiled_images/Index.idx.xml')

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


In [46]:
metadata

,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0305K1F1P1R1,Ok,r03c05f01p01-ch1sk1fk1fl1.tiff,3,5,1,1,0,1,1,...,0,0.135621503,0,2021-04-16T19:11:18.61+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0305K1F1P1R2,Ok,r03c05f01p01-ch2sk1fk1fl1.tiff,3,5,1,1,0,2,1,...,0,0.135621503,0,2021-04-16T19:11:18.61+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0305K1F1P2R1,Ok,r03c05f01p02-ch1sk1fk1fl1.tiff,3,5,1,2,0,1,1,...,2E-06,0.1356235,0,2021-04-16T19:11:18.89+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0305K1F1P2R2,Ok,r03c05f01p02-ch2sk1fk1fl1.tiff,3,5,1,2,0,2,1,...,2E-06,0.1356235,0,2021-04-16T19:11:18.89+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0305K1F1P3R1,Ok,r03c05f01p03-ch1sk1fk1fl1.tiff,3,5,1,3,0,1,1,...,4E-06,0.135625601,0,2021-04-16T19:11:19.17+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,0305K75F9P1R2,Ok,r03c05f09p01-ch2sk75fk1fl1.tiff,3,5,9,1,74,2,1,...,0,0.135624304,266400.56,2021-04-19T21:11:30.447+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4045,0305K75F9P2R1,Ok,r03c05f09p02-ch1sk75fk1fl1.tiff,3,5,9,2,74,1,1,...,2E-06,0.135626301,266400.56,2021-04-19T21:11:30.713+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4046,0305K75F9P2R2,Ok,r03c05f09p02-ch2sk75fk1fl1.tiff,3,5,9,2,74,2,1,...,2E-06,0.135626301,266400.56,2021-04-19T21:11:30.713+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4047,0305K75F9P3R1,Ok,r03c05f09p03-ch1sk75fk1fl1.tiff,3,5,9,3,74,1,1,...,4E-06,0.135628298,266400.56,2021-04-19T21:11:30.993+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


In [50]:
image_dir = '../data/untiled_images/'
images = tile.compile_mosaic(image_dir, metadata, row=3, col=5,  n_tile_cols=3, n_tile_rows=3)

Expected reshape: (75, 2, 3, 1198, 1198)


In [53]:
images

dask.array<reshape, shape=(75, 2, 3, 1198, 1198), dtype=uint16, chunksize=(1, 1, 1, 428, 428), chunktype=numpy.ndarray>

In [51]:
loaded_images = images.compute().compute() # why do i need to compute twice?

In [55]:
loaded_images.shape

(75, 2, 3, 1198, 1198)

In [32]:
import napari

In [57]:
viewer = napari.Viewer(title = 'Fully loaded mosaic compilation')
viewer.add_image(loaded_images, channel_axis =1, colormap=['green', 'magenta'])

[<Image layer 'Image' at 0x7f62b821cd90>,
 <Image layer 'Image [1]' at 0x7f62b42c9e10>]

## Optional: Save out fully compiled mosaic image array:

### 1. As Zarr

In [67]:
import numpy as np
import zarr

# Open root Zarr store
z = zarr.open('../data/example_data.zarr', mode='w')
img_grp = z.create_group('images')             # Create 'images/' group
img_grp.create_dataset('0',                    # Create dataset at 'images/0'
    data=loaded_images,
    chunks=(1, 1, 1, 256, 256),
    compressor=None
)


<zarr.core.Array '/images/0' (75, 2, 3, 1198, 1198) uint16>

##### Append some metadata

In [68]:
# Calculate pixel sizes
res_x = float(metadata['ImageResolutionX'].astype(float).iloc[0])
res_y = float(metadata['ImageResolutionY'].astype(float).iloc[0])
z_vals = metadata['PositionZ'].astype(float).unique()
res_z = float(np.ptp(z_vals) / (len(z_vals) - 1)) if len(z_vals) > 1 else 1.0

# Attach metadata to root
z.attrs.update({
    "multiscales": [{
        "version": "0.4",
        "axes": [{"name": a, "type": "time" if a == "t" else "space"} for a in "tczyx"],
        "datasets": [{"path": "images/0", "coordinateTransformations": [{
            "type": "scale", "scale": [1.0, 1.0, res_z, res_y, res_x]
        }]}]
    }],
    "description": "Example time-lapse of Mtb-infected macrophages",
    "axes": "TCZYX",
    "unit": "micrometer",
    "pixel_size": {"X": res_x, "Y": res_y, "Z": res_z},
    "n_timepoints": loaded_images.shape[0],
    "n_channels": loaded_images.shape[1],
    "n_zplanes": loaded_images.shape[2],
    "harmony_metadata": {
        "Channels": metadata['ChannelName'].unique().tolist(),
        "ObjectiveNA": float(metadata['ObjectiveNA'].astype(float).max()),
        "ExposureTime (ms)": float(metadata['ExposureTime'].astype(float).mean())
    }
})


### 2. As TIFF

In [ ]:
import tifffile as tiff

# Save to file
tiff.imwrite(
    '../data/example_data.ome.tiff',
    loaded_images,  # shape (T, C, Z, Y, X)
    photometric='minisblack',
    metadata={'axes': 'TCZYX'},
    bigtiff=True 
)